# [ポリシー](https://www.tensorflow.org/agents/tutorials/3_policies_tutorial)

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

from tf_agents.specs import array_spec
from tf_agents.specs import tensor_spec
from tf_agents.networks import network

from tf_agents.policies import py_policy
from tf_agents.policies import random_py_policy
from tf_agents.policies import scripted_py_policy

from tf_agents.policies import tf_policy
from tf_agents.policies import random_tf_policy
from tf_agents.policies import actor_policy
from tf_agents.policies import q_policy
from tf_agents.policies import greedy_policy

from tf_agents.trajectories import time_step as ts

## Pythonポリシー

このインターフェースは`policies/py_policy.PyPolicy`に定義されている。

最も重要なメソッドは、環境からの観測を含む`time_step`を、次の属性を含む`PlicyStep`にマップする`action(time_step)`です。

- `action` ：環境に適用されるアクション。
- `state` ：次のアクション呼び出しに供給されるポリシーの状態（RNN状態など）。
- `info` ：アクションログの確率などのオプションのサイド情報。

`time_step_spec`と`action_spec`は、入力タイムステップと出力アクションの仕様を返す。

ポリシーには、ステートフルポリシーの状態をリセットするために通常使用される`reset`機能もあります。

`update(new_policy)`関数は、 `self`を`new_policy`に向けて更新します。

### 例1：ランダムなPythonポリシー

入力された`time_step`は無視されます。

In [2]:
action_spec = array_spec.BoundedArraySpec((2,), np.int32, -10, 10)
my_random_py_policy = random_py_policy.RandomPyPolicy(time_step_spec=None,
    action_spec=action_spec)
time_step = None
action_step = my_random_py_policy.action(time_step)
print(action_step)
action_step = my_random_py_policy.action(time_step)
print(action_step)

PolicyStep(action=array([ 1, -5], dtype=int32), state=(), info=())
PolicyStep(action=array([4, 1], dtype=int32), state=(), info=())


### 例2：スクリプト化されたPythonポリシー

スクリプト化されたポリシーは、 `(num_repeats, action)`タプルのリストとして表されるアクションのスクリプトを再生します。 `action`関数が呼び出されるたびに、指定された回数の繰り返しが完了するまでリストから次のアクションが返され、リスト内の次のアクションに進みます。 `reset`メソッドを呼び出して、リストの最初から実行を開始できます。

In [3]:
action_spec = array_spec.BoundedArraySpec((2,), np.int32, -10, 10)
action_script = [(1, np.array([5, 2], dtype=np.int32)), 
                 (0, np.array([0, 0], dtype=np.int32)), # Setting `num_repeats` to 0 will skip this action.
                 (2, np.array([1, 2], dtype=np.int32)), 
                 (1, np.array([3, 4], dtype=np.int32))]

my_scripted_py_policy = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=None, action_spec=action_spec, action_script=action_script)

policy_state = my_scripted_py_policy.get_initial_state()
time_step = None
print('Executing scripted policy...')
action_step = my_scripted_py_policy.action(time_step, policy_state)
print(action_step)
action_step= my_scripted_py_policy.action(time_step, action_step.state)
print(action_step)
action_step = my_scripted_py_policy.action(time_step, action_step.state)
print(action_step)

print('Resetting my_scripted_py_policy...')
policy_state = my_scripted_py_policy.get_initial_state()
action_step = my_scripted_py_policy.action(time_step, policy_state)
print(action_step)

Executing scripted policy...
PolicyStep(action=array([5, 2], dtype=int32), state=[0, 1], info=())
PolicyStep(action=array([1, 2], dtype=int32), state=[2, 1], info=())
PolicyStep(action=array([1, 2], dtype=int32), state=[2, 2], info=())
Resetting my_scripted_py_policy...
PolicyStep(action=array([5, 2], dtype=int32), state=[0, 1], info=())


## TensorFlowポリシー

TensorFlowポリシーは、Pythonポリシーと同じインターフェースに従います。いくつかの例を見てみましょう。

### 例1：ランダムTFポリシー

RandomTFPolicyを使用して、特定の離散/連続`action_spec`に従ってランダムアクションを生成できます。入力された`time_step`は無視されます。

In [4]:
action_spec = tensor_spec.BoundedTensorSpec(
    (2,), tf.float32, minimum=-1, maximum=3)
input_tensor_spec = tensor_spec.TensorSpec((2,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)

my_random_tf_policy = random_tf_policy.RandomTFPolicy(
    action_spec=action_spec, time_step_spec=time_step_spec)
observation = tf.ones(time_step_spec.observation.shape)
time_step = ts.restart(observation)
action_step = my_random_tf_policy.action(time_step)

print('Action:')
print(action_step.action)

Action:
tf.Tensor([-0.7680621  -0.31236172], shape=(2,), dtype=float32)


### 例2：Actor policy

アクターポリシーは以下のいずれかのネットワークを使って作成することができる。

- `time_steps`をアクションにマッピングしするネットワーク
- `time_steps`をアクション上の確率分布にマッピングするネットワーク

#### Action networkを使用する

この例では、アクションテンソルを生成するアクションネットワークを使用してポリシーを作成する。この場合、`policy.distribution(time_step)`は`policy.action(time_step)`の出力に関する決定論的（デルタ）分布を返す。確率的ポリシーを生成したい場合は、アクションにノイズを追加するポリシーラッパーでアクターポリシーをラップする。

In [5]:
class ActionNet(network.Network):

    def __init__(self, input_tensor_spec, output_tensor_spec):
        super(ActionNet, self).__init__(
            input_tensor_spec=input_tensor_spec,
            state_spec=(),
            name='ActionNet')
        self._output_tensor_spec = output_tensor_spec
        self._sub_layers = [
            tf.keras.layers.Dense(action_spec.shape.num_elements(), activation=tf.nn.tanh),
        ]

    def call(self, observations, step_type, network_state):
        del step_type

        output = tf.cast(observations, dtype=tf.float32)
        for layer in self._sub_layers:
            output = layer(output)
        actions = tf.reshape(output, [-1] + self._output_tensor_spec.shape.as_list())

        # Scale and shift actions to the correct range if necessary.
        return actions, network_state

In [6]:
input_tensor_spec = tensor_spec.TensorSpec((4,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)
action_spec = tensor_spec.BoundedTensorSpec((3,), tf.float32, minimum=-1, maximum=1)

action_net = ActionNet(input_tensor_spec, action_spec)

my_actor_policy = actor_policy.ActorPolicy(
    time_step_spec=time_step_spec,
    action_spec=action_spec,
    actor_network=action_net)

In [7]:
# We can apply an arbitrary batch of `times_steps` to this actor policy
# (`time_steps` must follow the `time_step_spec`)

batch_size = 2
observations = tf.ones([2] + time_step_spec.observation.shape.as_list())
print('Observations:\n', observations)

time_step = ts.restart(observations, batch_size)
print('Initial time step:\n', time_step)

action_step = my_actor_policy.action(time_step)
print('Action:')
print(action_step.action)

distribution_step = my_actor_policy.distribution(time_step)
print('Action distribution:')
print(distribution_step.action)

Observations:
 tf.Tensor(
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]], shape=(2, 4), dtype=float32)
Initial time step:
 TimeStep(step_type=<tf.Tensor: shape=(2,), dtype=int32, numpy=array([0, 0], dtype=int32)>, reward=<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0., 0.], dtype=float32)>, discount=<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>, observation=<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)>)
Action:
tf.Tensor(
[[-0.41495764 -0.7019905   0.13005573]
 [-0.41495764 -0.7019905   0.13005573]], shape=(2, 3), dtype=float32)
Action distribution:
tfp.distributions.Deterministic("Deterministic", batch_shape=[2, 3], event_shape=[], dtype=float32)


#### Action distribution networkを使用する

これを使っても確率的ポリシーを生成することができる。

In [8]:
class ActionDistributionNet(ActionNet):

    def call(self, observations, step_type, network_state):
        action_means, network_state = super(ActionDistributionNet, self).call(
            observations, step_type, network_state)

        action_std = tf.ones_like(action_means)
        return tfp.distributions.MultivariateNormalDiag(action_means, action_std), network_state

In [9]:
action_distribution_net = ActionDistributionNet(input_tensor_spec, action_spec)

my_actor_policy = actor_policy.ActorPolicy(
    time_step_spec=time_step_spec,
    action_spec=action_spec,
    actor_network=action_distribution_net)

action_step = my_actor_policy.action(time_step)
print('Action:')
print(action_step.action)
distribution_step = my_actor_policy.distribution(time_step)
print('Action distribution:')
print(distribution_step.action)

Action:
tf.Tensor(
[[-1.         -0.266338   -0.15280703]
 [-0.26316485 -1.         -0.532077  ]], shape=(2, 3), dtype=float32)
Action distribution:
tfp.distributions.MultivariateNormalDiag("ActionNet_MultivariateNormalDiag", batch_shape=[2], event_shape=[3], dtype=float32)


アクションの値を与えた`action_spec`内であることを保証する場合は、`ActorPolicy`のコンストラクタで引数に`clip=True`を指定する（デフォルト）。そうしたくなければ、`clip=False`を指定する。

確率的ポリシーを決定論的ポリシーに変換したい場合は、`GreedyPolicy`ラッパーを使って実現できる。これは`stochastic_policy.distribution().mode()`をアクションとし、それらの決定論的（デルタ）分布を`distribution()`として選択する。

### 例3：Qポリシー

Qポリシーは、DQNなどのエージェントで使用され、個別のアクションごとにQ値を予測するQネットワークに基づいています。特定のタイムステップで、Qポリシーのアクション分布は、q値をロジットとして使用して作成されたカテゴリ分布です。

In [10]:
input_tensor_spec = tensor_spec.TensorSpec((4,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)
action_spec = tensor_spec.BoundedTensorSpec((),
                                            tf.int32,
                                            minimum=0,
                                            maximum=2)
num_actions = action_spec.maximum - action_spec.minimum + 1

In [11]:
class QNetwork(network.Network):

    def __init__(self, input_tensor_spec, action_spec, num_actions=num_actions, name=None):
        super(QNetwork, self).__init__(
            input_tensor_spec=input_tensor_spec,
            state_spec=(),
            name=name)
        self._sub_layers = [
            tf.keras.layers.Dense(num_actions),
        ]

    def call(self, inputs, step_type=None, network_state=()):
        del step_type
        inputs = tf.cast(inputs, tf.float32)
        for layer in self._sub_layers:
            inputs = layer(inputs)
        return inputs, network_state

In [12]:
batch_size = 2
observation = tf.ones([batch_size] + time_step_spec.observation.shape.as_list())
time_steps = ts.restart(observation, batch_size=batch_size)

my_q_network = QNetwork(
    input_tensor_spec=input_tensor_spec,
    action_spec=action_spec)
my_q_policy = q_policy.QPolicy(
    time_step_spec, action_spec, q_network=my_q_network)
action_step = my_q_policy.action(time_steps)
distribution_step = my_q_policy.distribution(time_steps)

print('Action:')
print(action_step.action)

print('Action distribution:')
print(distribution_step.action)

Action:
tf.Tensor([2 0], shape=(2,), dtype=int32)
Action distribution:
tfp.distributions.Categorical("Categorical", batch_shape=[2], event_shape=[], dtype=int32)


## ポリシーラッパー

ポリシーラッパーを使用して、特定のポリシーを変更できます（ノイズの追加など）。ポリシーラッパーはポリシー（Python / TensorFlow）のサブクラスであるため、他のポリシーと同じように使用できます。

### 例：Greedy policy
Greedyラッパーを使用して、 `distribution()`を実装しているTensorFlowポリシーをラップできます。
- `GreedyPolicy.action()`は`wrapped_policy.distribution().mode()`を返し、
- `GreedyPolicy.distribution()`は`GreedyPolicy.action()`上の決定論的/デルタ分布です

In [13]:
my_greedy_policy = greedy_policy.GreedyPolicy(my_q_policy)

action_step = my_greedy_policy.action(time_steps)
print('Action:')
print(action_step.action)

distribution_step = my_greedy_policy.distribution(time_steps)
print('Action distribution:')
print(distribution_step.action)

Action:
tf.Tensor([0 0], shape=(2,), dtype=int32)
Action distribution:
tfp.distributions.DeterministicWithLogProb("Deterministic", batch_shape=[2], event_shape=[], dtype=int32)
